In [ ]:
import numpy as np
from bqplot import *
from bqplot.marks import Graph
from ipywidgets import IntSlider, Dropdown, RadioButtons, HBox, VBox, Button, Layout
from bqplot import pyplot as plt
from bqplot import OrdinalScale

from IPython.display import display

In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

np.random.seed(7)
data_df_total = pd.read_csv('./data_files/credit-training.csv', index_col=0)
result_column = 'SeriousDlqin2yrs'

train_idx, test_idx = train_test_split(data_df_total.index.values, test_size=0.3,
                                       stratify=data_df_total[result_column])
train_data = data_df_total.loc[train_idx]
test_data = data_df_total.loc[test_idx]

In [ ]:
X_train = pd.read_csv('data_files/cleaned_train.csv')
X_test = pd.read_csv('data_files/cleaned_test.csv')

y_train = X_train['label']
y_test = X_test['label']

X_train = X_train.drop('label', axis=1)
X_test = X_test.drop('label', axis=1)

In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

np.random.seed(7)
data_df_total = pd.read_csv('./data_files/credit-training.csv', index_col=0)
result_column = 'SeriousDlqin2yrs'

train_idx, test_idx = train_test_split(data_df_total.index.values, test_size=0.3,
                                       stratify=data_df_total[result_column])
train_data = data_df_total.loc[train_idx]
test_data = data_df_total.loc[test_idx]

In [ ]:
from sklearn.metrics import roc_auc_score, precision_score, recall_score, accuracy_score
## Utility functions for model evaluation.
def eval_preds(y_true, y_probs, y_preds):
    return {'precision': precision_score(y_true, y_preds),
            'accuracy': accuracy_score(y_true, y_preds),
            'recall': recall_score(y_true, y_preds),
            'auc': roc_auc_score(y_true, y_probs)}

def get_model_eval(true_train, train_predictions, true_test=None, test_predictions=None):
    train_eval = eval_preds(true_train, *train_predictions)
    if true_test is None:
        return pd.Series(train_eval)
    else:
        test_eval = eval_preds(true_test, *test_predictions)
        return pd.DataFrame([train_eval, test_eval], index=['Train', 'Test'])

def probas_to_classes(probas):
    return (probas >= 0.5).astype(float)

In [ ]:
## Training the model.
num_epochs = 15
batch_size = 5000

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras import regularizers

import keras
import pandas as pd
import keras.backend as K

import tensorflow as tf
sess = tf.InteractiveSession()
K.set_session(sess)

sample_weights = np.ones(X_train.shape[0])

## callback to compute the gradients
class WeightsGradientsCallback(keras.callbacks.Callback):
    def __init__(self):
        self.gradients = []
        self.train_auc = []
        self.test_auc = []
        self.weights = []
    
    def on_epoch_end(self, epoch, logs={}):
        input_values = [X_train_norm, sample_weights, y_train.values.reshape(-1, 1), 0]
        gradient_values = compute_gradients(input_values)
        self.gradients.append(gradient_values)
        
        self.train_auc.append(roc_auc_score(y_train.values.flatten(), 
                                            self.model.predict(X_train_norm)))
        self.test_auc.append(roc_auc_score(y_test.values.flatten(), 
                                           self.model.predict(X_test_norm)))
        
        auc_line.x = np.arange(0, epoch + 1)
        auc_line.y = [self.train_auc, self.test_auc]
        
        weights = list(range(len(self.model.layers)))
        for i, l in enumerate(self.model.layers):
            weights[i] = l.get_weights()
        self.weights.append(weights)
        
class LayerEvalsCallsback(keras.callbacks.Callback):
    def __init__(self):
        self.train_activs = []
        self.test_activs = []
        
    def on_epoch_end(self, epoch, logs={}):
        train_activs = layer_evaluator([X_train_norm, 1.])
        test_activs = layer_evaluator([X_test_norm, 1.])
        
        self.train_activs.append(train_activs)
        self.test_activs.append(test_activs)

weights_callback = WeightsGradientsCallback()
activs_callback = LayerEvalsCallsback()

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

X_train_norm = scaler.fit_transform(X_train.values.astype('float'))
X_test_norm = scaler.transform(X_test.values.astype('float'))
dropout_prob = 0.2

model = Sequential()
model.add(Dense(20, input_dim=X_train_norm.shape[1], activation='relu'))
model.add(Dropout(dropout_prob))

model.add(Dense(10, activation='tanh'))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', 
              metrics=['accuracy'])

## variables for WeightsGradientsCallback:
gradients = model.optimizer.get_gradients(model.model.total_loss, 
                                          model.trainable_weights)
input_tensors = [model.model.inputs[0], model.model.sample_weights[0], 
                 model.model.targets[0], K.learning_phase()]
compute_gradients = K.function(inputs=input_tensors, outputs=gradients)   

## display figure for WeightsGradientsCallback
auc_fig = plt.figure(title='Train and Test AUC vs epoch', legend_location='top-left')
auc_line = plt.plot([0], [0], marker='circle', marker_size=32, colors=['DeepSkyBlue', 'Red'], 
                              labels=['Training', 'Test'], display_legend=True)
display(auc_fig)

## variables for LayerEvalsCallback
inp = model.input                                          
outputs = [layer.output for layer in model.layers]         
layer_evaluator = K.function([inp]+ [K.learning_phase()], outputs) 

# Fit the model
model.fit(X_train_norm, y_train.values, verbose=2,
          epochs=num_epochs, batch_size=batch_size,
           callbacks=[weights_callback, activs_callback])

train_probs = model.predict(X_train_norm).flatten()
train_preds = probas_to_classes(train_probs)

test_probs = model.predict(X_test_norm).flatten()
test_preds = probas_to_classes(test_probs)


model_eval = get_model_eval(y_train, [train_probs, train_preds],
                            y_test, [test_probs, test_preds])
print(model_eval)
sess.close()    

In [ ]:
def get_activations_hist(epoch, layer, node, data='Train'):
    if data == 'Train':
        activs_array = activs_callback.train_activs
    else:
        activs_array = activs_callback.test_activs
    
    if layer >= 1:
        layer = layer + 1
    layer_activs = activs_array[epoch - 1][layer].T[node]
    return layer_activs

In [ ]:
def get_cleaned_weights(weights_mat):
    weights_ret = []
    for w in weights_mat:
        if np.shape(w)[0] == 0:
            # this is a dropout layer or a reg layer which does no have weights
            pass
        else:
            weights_ret.append(w)
    return weights_ret

def get_weights_for_node_at_layer(weights, epoch_num, layer_num, node_num):
    # max_layers = len(weights)
    layer_params = weights[epoch_num][layer_num]
    
    layer_weights = layer_params[0]
    layer_bias = layer_params[1]
    
    node_weights = layer_weights[:, node_num]
    node_bias = layer_bias[node_num]
    
    return (node_bias, node_weights)

def get_gradients_for_node_at_layer(gradients, epoch_num, layer_num, node_num):
    layer_gradients = gradients[epoch_num][2 * layer_num]
    layer_bias_gradients = gradients[epoch_num][2 * layer_num + 1]
    
    node_gradients = layer_gradients[:, node_num]
    node_bias_gradiens = layer_bias_gradients[node_num]
    
    return(node_bias_gradiens, node_gradients)

cleaned_weights = []

for w in weights_callback.weights:
    cleaned_weights.append(get_cleaned_weights(w))

In [ ]:
from neural_net import NeuralNet

In [ ]:
nn = NeuralNet(num_inputs=12, num_hidden_layers=[20, 10], num_outputs=1)

epoch_slider = IntSlider(description='Epoch:', min=1, max=num_epochs, value=1)
mode_dd = Dropdown(description='View', options=['Weights', 'Gradients', 'Activations'], value='Weights')
update_btn = Button(description='Update')

bar_figure = plt.figure()
bar_plot = plt.bar([], [], scales={'x': OrdinalScale()})

hist_figure = plt.figure(title='Histogram of Activations')
hist_plot = plt.hist([], bins=20)

controls = HBox([epoch_slider, mode_dd, update_btn])
nn.graph.tooltip = bar_figure

In [ ]:
def update_bar_chart(layer, node):
    epoch = epoch_slider.value
    
    if mode_dd.value == 'Activations':
        display_vals = get_activations_hist(epoch, layer-1, node)
        hist_plot.sample = display_vals
        nn.graph.tooltip = hist_figure
        hist_figure.title = mode_dd.value + ' for layer:' + str(layer) + ' node: ' + str(node) + ' at epoch: ' + str(epoch)
        return

    if mode_dd.value == 'Weights':
        display_vals = get_weights_for_node_at_layer(cleaned_weights, epoch, layer-1, node)
    elif mode_dd.value == 'Gradients':
        display_vals = get_gradients_for_node_at_layer(weights_callback.gradients, epoch, layer-1, node)
        
    return_vals = np.append([display_vals[0]], display_vals[1])
    
    bar_figure.title = mode_dd.value + ' for layer:' + str(layer) + ' node: ' + str(node) + ' at epoch: ' + str(epoch)
    bar_plot.x = np.arange(len(return_vals))
    bar_plot.y = return_vals
    
node_counts = [nn.num_inputs] + nn.num_hidden_layers + [nn.nodes_output_layer]

def hovered_change(change):
    point_index = change['new']
    set_colors(point_index)
    
    if point_index is None:
        return
    else:
        for i, n in enumerate(node_counts):
            if point_index < n:
                break
            else:
                point_index = point_index - n
        if i > 0:
            update_bar_chart(i, point_index)
            
def set_colors(index):
    link_data_new = []
    count = 0
    for v in nn.graph.link_data:
        v_new = {}
        v_new['source'] = v['source']
        v_new['target'] = v['target']
        if v['target'] == index:
            v_new['value'] = (count % 11) + 1
            count = count + 1
        else:
            v_new['value'] = 0
        link_data_new.append(v_new)
    nn.graph.link_data = link_data_new
        
    
nn.graph.observe(hovered_change, 'hovered_point')

VBox([controls, nn], layout=Layout(min_height='1000px'))    